# Project 1: Language Modeling and Fake News Classification

Names: Yuanhong Cao, Zeyu Chen

Netids: yc2668, zc436

**After you make your own copy, please rename this notebook by clicking on it's name in the upper left corner.** It should be named: 4740_FA20_p1_netid1_netid2

Don't forget to share your newly copied notebook with your partner!

**Reminder: both of you can't work in this notebook at the same time from different computers/browser windows because of sync issues. We even suggest to close the tab with this notebook when you are not working on it so your partner doesn't get sync issues.**

## Introduction
In this project we will build an **n-gram-based language model** for fake news classification. We will also investigate a feature-based **Naive Bayes model**. The task we are faced with is to **decide whether a news article is fake or real**. While some fake articles are so absurd and nonsensical that one can clearly guess they are fake, most fake news is actually quite hard to detect. [Various studies](https://pacscenter.stanford.edu/research/program-on-democracy-and-the-internet/deception-detection-accuracy-for-fake-news-headlines-on-social-media/) have shown that most people can have an error rate up to 50% depending on the theme of the article. 

To help us approach this problem, we will use NLP techniques covered thus far to frame this as a (supervised) binary classification task, where each article will have a label $y \in \{0,1\}$, where *0 indicates a fake article* and *1 indicates a real one*. You will train and validate your two different models and then run them on a test data set with hidden $y$ labels. You will then submit the results on the test data set to Kaggle to participate in our class-wide competition!

The project is divided into six parts:
1. Dataset loading and preprocessing
2. Unsmoothed n-gram language model (LM): build the unsmoothed n-gram language model using our Fake News corpus. 
3. Smoothed n-gram language model: build a smoothed version of the model from part 2.
4. Perplexity: compute perplexity for both the unsmoothed and smoothed model
5. Putting everything together and submitting the first model to Kaggle
6. Naive Bayes: build a feature-based Naive Bayes model to perform the same classification task. Compare the LM with Naive Bayes and identify the pros and cons of each.

**Logistics:** You should work in **groups of 2 students**. Students in the same group will get the same grade. Thus, you should make sure that everyone in your group contributes to the project. 
- **Remember to form groups on BOTH CMS and Gradescope** or not all group members will receive grades. You can use the Teammate Search option on Piazza to find a partner for this project.

**Advice:** Please complete the written parts of this notebook in a clear and informative way. This is where you get to show us that you understand not only what you are doing but also why and how you are doing it. So be clear, organized and concise; avoid vagueness and excess verbiage. Spend time
doing error analysis for the models. This is how you understand the advantages and drawbacks of the systems you build. 
- It's also useful to think about how the theory of n-grams/Naive Bayes bridges with the real world application we are building. Think about what you expect from these models based on your current understanding, and then see if your expectation aligns with empirical results that you'll get. 

## General Guidelines
In this project, we provide a few code snippets or starter points in case you need them. You DO NOT need to follow the structure. 

If you think you have a better idea, go for it. You can ADD, MODIFY, or DELETE any code snippets given to you.

**Let's do this** 🚀

### Dataset

You are given a **News Corpus** on CMS, which consists of roughly the same amount of real and fake news articles.

Real news example:
```
The OpenAI technology, known as GPT-2, is designed to predict the next word given all the previous words it is shown within some text. The language-based model has been trained on a dataset of 8 million web pages.
```

Fake news example:
```
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
```

In the dataset folder you should find 4 files, training and validation splits for both real and fake news.

The project will proceed generally as follows in terms of code development:
1. Write code to train unsmoothed unigram and bigram language models for an arbitrary corpus
2. Implement smoothing and unknown word handling. 
3. Implement the Perplexity calculation. 
4. Using 1, 2 and 3, together with the provided training and validation sets, develop a language-model-based approach for Fake News Classification.
5. Apply your best language-model-based news classifier (from 4) to the
provided test set. Submit the results to the online Kaggle competition. 
6. Use any existing implementation of Naive Bayes (and the provided training and validation sets) to create an additional Naive Bayes fake news classifier. Apply your best NB classifier to the provided test set. Submit the results to the separate Kaggle competition (for NB classifiers). 

We will progress towards these tasks throughout this notebook.

# Part 1: Preprocessing the Dataset
In this part, you are going to do a few things:
* Connect to the google drive where the data set is stored
* Load and read files
* Preprocess the text

------
**Please upload the dataset to each partner's individual Google Drive now.** We suggest using the same folder structure within Google Drive because the notebook is shared among you, so the code to load the data would have to be changed every time if folder structures are different. One folder structure might be: Google Drive/CS 4740/Project 1/Dataset/ or whatever works for you. See our code below for an example of how we load the data from Google Drive.

## 1.1 Connect to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## 1.2 Load and read files
First, let's install [NLTK](https://www.nltk.org/), a very widely package for NLP preprocessing (and other tasks) for Python.

In [ ]:
!pip install -U nltk

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.5)


Then we read and load data.

In [ ]:
import os
import io
from nltk import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

root_path = os.path.join(os.getcwd(), "drive", "My Drive/CS 4740/Project 1/Dataset/") # replace based on your Google drive organization
dataset_path = os.path.join(root_path, "real_fake") # same here

with io.open(os.path.join(dataset_path, "trueDataTrain.txt"), encoding='utf8') as real_file:
  real_news = real_file.read()
with io.open(os.path.join(dataset_path, "trueDataValidation.txt"), encoding='utf8') as real_file:
  real_news_validation = real_file.read()
with io.open(os.path.join(dataset_path, "fakeDataTrain.txt"), encoding='utf8') as fake_file:
  fake_news = fake_file.read()
with io.open(os.path.join(dataset_path, "fakeDataValidation.txt"), encoding='utf8') as real_file:
  fake_news_validation = real_file.read()

tokenized_real_news_training = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(real_news)]
tokenized_fake_news_training = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(fake_news)]
tokenized_real_news_validation = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(real_news_validation)]
tokenized_fake_news_validation = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(fake_news_validation)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sanity checks for our real and fake training sets

In [ ]:
tokenized_real_news_training[0]

['london',
 '(',
 'reuters',
 ')',
 '-',
 'britain',
 'is',
 'seeking',
 'to',
 'build',
 'on',
 'the',
 'recent',
 'momentum',
 'in',
 'the',
 'brexit',
 'divorce',
 'talks',
 'with',
 'the',
 'european',
 'union',
 'before',
 'a',
 'summit',
 'next',
 'month',
 ',',
 'a',
 'spokeswoman',
 'for',
 'britain',
 's',
 'department',
 'for',
 'exiting',
 'the',
 'european',
 'union',
 'said',
 'on',
 'tuesday',
 '.']

In [ ]:
tokenized_fake_news_training[0]

['rick',
 'santorum',
 'had',
 'the',
 'impossible',
 'job',
 'of',
 'defending',
 'donald',
 'trump',
 'during',
 'friday',
 's',
 'real',
 'time',
 'and',
 'he',
 'found',
 'himself',
 'humiliatingly',
 'outnumbered.bill',
 'maher',
 'began',
 'by',
 'expressing',
 'astonishment',
 'at',
 'how',
 'the',
 'republican',
 'nominee',
 'has',
 'been',
 'acting',
 'throughout',
 'the',
 'campaign',
 '.']

## 1.3 Data Preprocessing & Preparation

There's a well-known parable in machine learning that 80% of the work is all about data preparation, 10% is supporting infrastructure and 10% is actual modeling. If your "raw" dataset is not preprocessed and prepared in a way to maximize its value, then your model will be more like this: https://xkcd.com/1838/. For this project, modeling is the star of the show for learning purposes, but we still want you to pay attention to the preprocessing stage.

*We've already tokenized and lowercased* the raw data for you. Here are a few extra things you might want to do:

- Think about edge cases. For example, you don't want to accidentally append a period to the last word of a sentence. 
- Watch out for apostrophes and other tricky things like quotations, they cause lots of edge cases. For example, "they're" can be all one token, or two tokens ("they", "'re") or even three tokens ("they", " ' ", "re"). 

Why did we lowercase all tokens? Because the computer will otherwise consider "The" and "the" as two separate words and this will cause problems.

Note that you may use existing
tools just for the purpose of preprocessing. 

Advice: don't get bugged down in the dozens of preprocessing packages and suggestions that you can find on Towards Data Science or Stack Overflow. Start with this [NLTK tutorial](https://lost-contact.mit.edu/afs/cs.pitt.edu/projects/nltk/docs/tutorial/introduction/nochunks.html#:~:text=The%20Natural%20Language%20Toolkit%20(NLTK,tokenization%2C%20tagging%2C%20and%20parsing.) and that should be plenty.

In [ ]:
for sent in tokenized_real_news_training:
  if sent[0] != '<s>':
    sent.insert(0, '<s>')
for sent in tokenized_fake_news_training:
  if sent[0] != '<s>':
    sent.insert(0, '<s>')

for s in range(len(tokenized_real_news_training)):
  if len(tokenized_real_news_training[s][-1]) > 1 and '.' in tokenized_real_news_training[s][-1]:
    tokenized_real_news_training[s][-1] = tokenized_real_news_training[s][-1][:-1]
    tokenized_real_news_training[s].extend('.')
for s in range(len(tokenized_fake_news_training)):
  if len(tokenized_fake_news_training[s][-1]) > 1 and '.' in tokenized_fake_news_training[s][-1]:
    tokenized_fake_news_training[s][-1] = tokenized_fake_news_training[s][-1][:-1]
    tokenized_fake_news_training[s].extend('.')

real_training_combined = [y for x in tokenized_real_news_training for y in x]
fake_training_combined = [y for x in tokenized_fake_news_training for y in x]

real_para_list = real_news_validation.splitlines()
for para in range(len(real_para_list)):
  real_para_list[para] = ['startsymbol ' + sub for sub in sent_tokenize(real_para_list[para])] 
  real_para_list[para] = ' '.join(real_para_list[para])
real_validation_sep = [list(map(str.lower, word_tokenize(sent))) for sent in real_para_list]
for para in range(len(real_validation_sep)):
  for w in range(len(real_validation_sep[para])):
    if real_validation_sep[para][w] == 'startsymbol':
      real_validation_sep[para][w] = '<s>'

fake_para_list = fake_news_validation.splitlines()
for para in range(len(fake_para_list)):
  fake_para_list[para] = ['startsymbol ' + sub for sub in sent_tokenize(fake_para_list[para])] 
  fake_para_list[para] = ' '.join(fake_para_list[para])
fake_validation_sep = [list(map(str.lower, word_tokenize(sent))) for sent in fake_para_list]
for para in range(len(fake_validation_sep)):
  for w in range(len(fake_validation_sep[para])):
    if fake_validation_sep[para][w] == 'startsymbol':
      fake_validation_sep[para][w] = '<s>'

tokenized_real_news_training[0]

['<s>',
 'london',
 '(',
 'reuters',
 ')',
 '-',
 'britain',
 'is',
 'seeking',
 'to',
 'build',
 'on',
 'the',
 'recent',
 'momentum',
 'in',
 'the',
 'brexit',
 'divorce',
 'talks',
 'with',
 'the',
 'european',
 'union',
 'before',
 'a',
 'summit',
 'next',
 'month',
 ',',
 'a',
 'spokeswoman',
 'for',
 'britain',
 's',
 'department',
 'for',
 'exiting',
 'the',
 'european',
 'union',
 'said',
 'on',
 'tuesday',
 '.']

**Q1.1: Show some observations or statistics from the dataset** (should be quantitative – i.e. most frequent words, most frequent bigram, etc.)

Your answer: For real_news file, the most frequent word is "the", which occurs 338510 times. The most frequent bigram is ". \<s>", which occurs 219300 times.

In [ ]:
#nltk.download('all')
from nltk.book import *

fdist1 = FreqDist(real_training_combined)
print(fdist1.most_common(10))

bgs = bigrams(real_training_combined)
fdist2 = FreqDist(bgs)
print(fdist2.most_common(10))

[('the', 338510), (',', 285270), ('<s>', 221017), ('.', 219494), ('to', 172440), ('of', 143661), ('a', 139118), ('and', 127475), ('in', 126832), ('on', 76214)]
[(('.', '<s>'), 219300), (('’', 's'), 38516), (('<s>', 'the'), 33798), (('of', 'the'), 33470), (('in', 'the'), 28822), (('<s>', '“'), 19170), ((',', 'the'), 18819), ((',', '”'), 17220), (('to', 'the'), 15594), (('said', '.'), 15158)]


**Please answer the following question**:

**Q1.2: What did you do in your preprocessing part?**

Example answer format:

A: We tokenized and lowercased all the words.

Your answer: We add start sentence marker at the beginning of every sentence and seperate all the periods maker after the last word of sentence in the training files.

# Part 2: Compute Unsmoothed Language Models.

To start, you will write a program that computes unsmoothed unigram and bigram probabilities. You should consider real and fake news as separate corpora and
generate a separate language model for each set of news.
We have already loaded the data and (partially) preprocessed it and you probably did some of your own preprocessing. 

Note that you were allowed to use existing
tools for the purpose of preprocessing, but you must write the code for gathering n-gram counts and computing n-gram probabilities yourself. 

For example, consider the
simple corpus consisting of the sole sentence:


> the students liked the assignment

Part of what your program would compute for a unigram and bigram model, for example,
would be the following:


> $P("the") = 0.4; P("liked") = 0.2; P("the"|"liked") = 1.0; P("students"|"the") = 0.5$

Remember to add a symbol to mark the beginning of sentence. See Sept. 7th lecture, p25-28 for an example.




**Advice**: jupyter notebooks (including colab) can be a double-edged sword. It's amazing and liberating to just start writing code and run it by simply running a cell. However, it gets messy very quickly. So, once you're done prototyping, we highly recommend using functions (classes may be unnecessary but go for it if you want) to make things cleaner and easier to debug.

## 2.1 Unsmoothed Uni-gram Model.

In this part of the project, you are trying to compute the probabilities for a unigram model. You might want to take in a list of words, and return the probabilities for each
occurence. Think of an efficient data structure to use here given what ratio of reads and puts you expect.

Please look at the example above and consider how we get the probabilities.

Below is a starter point you can go from, but you DO NOT need to stick it. Feel free to use your own design.

In [ ]:
"""
Reference code for start. You do not need to follow this.
Function [unsmoothed_unigram] computes the probabilities for a unigram model
lst: a list of words in a sentence
Return: [data structure of your choice] that stores the result
"""
import collections
def unsmoothed_unigram(lst):
  model = collections.defaultdict(lambda: 0)
  for w in lst:
    try:
      if w == '<s>':
        continue
      model[w] += 1
    except KeyError:
      model[w] = 1
      continue
  denominator = sum(model.values())
  for w in model:
    model[w] = model[w]/denominator
  return model

lst = ['<s>', 'the', 'students', 'liked', 'the', 'assignment']
result = unsmoothed_unigram(lst)
result

defaultdict(<function __main__.unsmoothed_unigram.<locals>.<lambda>>,
            {'assignment': 0.2, 'liked': 0.2, 'students': 0.2, 'the': 0.4})

## 2.2 Unsmoothed Bi-gram Model.

In this part of the project, you are trying to compute the probabilities for a bigram model. You can approach this with similar methods as above.

Remember the definition:
$p(w_n\mid w_{n-1})=\frac{C(w_{n-1}w_n)}{C(w_{n-1})}$ this means you might want to store two things (count of $w_{n-1}$ and count of $w_{n-1}w_n$).

In [ ]:
def unsmoothed_bigram(lst):
  bilst = list(bigrams(lst))
  model = collections.defaultdict(lambda: 0)
  for w in bilst:
    try:
      model[w] += 1
    except KeyError:
      model[w] = 1
      continue
  for w in model:
    denominator = lst.count(w[0])
    model[w] = model[w]/denominator
  return model

lst = ['<s>', 'the', 'students', 'liked', 'the', 'assignment']
result = unsmoothed_bigram(lst)
result

defaultdict(<function __main__.unsmoothed_bigram.<locals>.<lambda>>,
            {('<s>', 'the'): 1.0,
             ('liked', 'the'): 1.0,
             ('students', 'liked'): 1.0,
             ('the', 'assignment'): 0.5,
             ('the', 'students'): 0.5})

**Please answer the following question**:

**Q2: What data structure are you using to store probabilities for unigrams and bigrams? Why did you select this data structure?**

Your answer: We used dictionary to store the probability because dictionary is a convenient strucure to look up values when we know the keys.

# Part 3: Smoothed Language Model
In this part, you will need to implement **at least one** smoothing method and **at least one** method to handle unknown words in the test data. You can choose any method(s) that you want for each. You should make clear
**what method(s)** were selected and **why**, providing a description for any non-standard approach (e.g., an approach that was not covered in class or in the readings). 

You should use the
provided validation sets to experiment with different smoothing/unknown word handling
methods if you wish to see which one is more effective for this task. (We will cover this in Part 4).

## 3.1 Unknown Words Handling

**Please answer the following questions:**

**Q3.1: How are you going to handle unknown words? What parameters might be needed? Do you need a method to determine the value?**

Your answer: We select a vocabulary size V (top 70% of the most frequent words) in advance and choose the top V words by frequency and replace the rest by UNK. The parameter is the input list and we do not need other methods.


In [ ]:
def unknown_handling(lst):
  fdist = FreqDist(lst)
  top70 = [word for (word, _) in fdist.most_common(int(len(set(lst)) * 0.7))]
  mapping = collections.defaultdict(lambda: 'UNK')
  for w in top70:
    mapping[w] = w
  filtered = [mapping[w] for w in lst]
  return filtered

filtered = unknown_handling(real_training_combined[:50])
filtered

['<s>',
 'london',
 '(',
 'reuters',
 ')',
 '-',
 'britain',
 'is',
 'seeking',
 'to',
 'build',
 'on',
 'the',
 'recent',
 'momentum',
 'in',
 'the',
 'brexit',
 'divorce',
 'talks',
 'with',
 'the',
 'european',
 'union',
 'before',
 'a',
 'summit',
 'next',
 'month',
 'UNK',
 'a',
 'UNK',
 'for',
 'britain',
 'UNK',
 'UNK',
 'for',
 'UNK',
 'the',
 'european',
 'union',
 'UNK',
 'on',
 'UNK',
 'UNK',
 '<s>',
 'UNK',
 'UNK',
 'UNK',
 'UNK']

## 3.2 Smoothing

In this part of project, we are going to compute the probabilities for unigram and bigram models after smoothing.
There are several smoothing methods you can start with:
* add-k
* Kneser-Ney
* Good-Turing
* ...

You need to compute for both unigram and bigram models.

Below is a starter point using add-k smoothing. As always, you DO NOT need to follow it; you DO NOT need to use add-k smoothing if you do not want to. You can pick ANY smoothing method you want.

In [ ]:
"""
Reference code for add-k smoothing on unigram model.
lst: a list of words in a sentence
k: parameter k for smoothing
Return: a dictionary of results after smoothing
"""
def add_k_unigram(lst, k):
  model = collections.defaultdict(lambda: 0)
  for w in lst:
    try:
      model[w] += 1
    except KeyError:
      model[w] = 1
      continue
  denominator = sum(model.values()) + len(model) * k
  for w in model:
    model[w] = (model[w] + k) / denominator
  return model

lst = ['<s>', 'i', 'wonder', 'how', 'i', 'wonder', 'why']
print(unknown_handling(lst))
result = add_k_unigram(unknown_handling(lst), 2)
result


['<s>', 'i', 'wonder', 'UNK', 'i', 'wonder', 'UNK']


defaultdict(<function __main__.add_k_unigram.<locals>.<lambda>>,
            {'<s>': 0.2,
             'UNK': 0.26666666666666666,
             'i': 0.26666666666666666,
             'wonder': 0.26666666666666666})

In [ ]:
"""
Reference code for add-k smoothing on bigram model.
lst: a list of words in a sentence
k: parameter k for smoothing
Return: a dictionary of results after smoothing
"""
def add_k_bigram(lst, k):
  uni_dic = collections.defaultdict(lambda: 0)
  for w in lst:
    try:
      uni_dic[w] += 1
    except KeyError:
      uni_dic[w] = 1
      continue

  bilst = list(bigrams(lst))
  bi_dic = collections.defaultdict(lambda: 0)
  for w in bilst:
    try:
      bi_dic[w] += 1
    except KeyError:
      bi_dic[w] = 1
      continue
  unique = len(uni_dic)
  result = bi_dic
  for w in bi_dic:
    numerator = bi_dic[w] + k
    denominator = uni_dic[w[0]] + unique * k
    result[w] = numerator/denominator
  return result

lst = ['<s>', 'i', 'wonder', 'how', 'i', 'wonder', 'why']
print(unknown_handling(lst))
result = add_k_bigram(unknown_handling(lst), 2)
result

['<s>', 'i', 'wonder', 'UNK', 'i', 'wonder', 'UNK']


defaultdict(<function __main__.add_k_bigram.<locals>.<lambda>>,
            {('<s>', 'i'): 0.3333333333333333,
             ('UNK', 'i'): 0.3,
             ('i', 'wonder'): 0.4,
             ('wonder', 'UNK'): 0.4})

**Please answer the following question:**

**Q3.2: Which smoothing method did you choose? Are there any parameters, if so how are you planning to pick the value? If you choose to implement more than 1 method (not a requirement), please state each of them. Providing a description for any non-standard approach, e.g., an approach that was not covered in class or in the readings**

Your answer: We choose add-k method and randomly choose k as 2. Basically we follow the reference code and use input dictionary to get the ouput dictionary in which there are probabilities of every token/bigram.


# Part 4: Perplexity
At this point, we have developed several language models: unigram vs bigram, unsmoothed vs smoothed. We now want to compare all the models. 

Implement code to compute the perplexity of a **“development set.”** (“Development set”
is just another way to refer to the validation set—part of a dataset that is distinct from
the training portion and the test portion.) Compute and report the perplexity of each
of the language models (one trained on true news and fake news) on
the development corpora. Compute perplexity as follows:
\begin{align*}
PP &= \left(\prod_i^N\frac{1}{P\left(W_i\mid W_{i-1}, ...W_{i-n+1}\right)}\right)^{\frac{1}{N}}\\
&=\exp \frac{1}{N}\sum_{i}^N-\log P\left(W_i\mid W_{i-1}, ...W_{i-n+1}\right)
\end{align*}
where $N$ is the total number of tokens in the test corpus and $P\left(W_i\mid W_{i-1}, ...W_{i-n+1}\right)$
is the n-gram probability of your model. Under the second definition above, perplexity
is a function of the average (per-word) log probability: use this to avoid numerical
computation errors.

Please complete the following tasks and report what you have observed. Remember, lower perplexity means better model.

## Task 1: Compute perplexity for smoothed unigram and smoothed bigram. 
*Note: If you choose more than one smoothing method, pick one of them to compute. If you need to try different values of parameters, you can try them out here.*


In [ ]:
# compute perplexity for one smoothing method on unigram, and one smoothing method on bigram.
import math

def perplexity(lst, model, mode):
  if mode == 2:
    lst = list(bigrams(lst))
  if len(lst) == 0:
    return 0
  perplexity = 0
  N = 0
  for w in lst:
    N += 1
    if w in model:
      perplexity -= math.log(model[w])
    elif mode == 1:
      perplexity -= math.log(model['UNK'])
    elif mode == 2:
      perplexity -= math.log(model[('UNK', 'UNK')])
  perplexity = math.exp(perplexity / N)
  return perplexity

real_model = add_k_unigram(unknown_handling(real_training_combined), 1)
real_model2 = add_k_bigram(unknown_handling(real_training_combined), 1)
fake_model = add_k_unigram(unknown_handling(fake_training_combined), 1)
fake_model2 = add_k_bigram(unknown_handling(fake_training_combined), 1)

real_validation_combined = [y for x in tokenized_real_news_validation for y in x]
fake_validation_combined = [y for x in tokenized_fake_news_validation for y in x]
print('perplexity of unigram model with real validation file', perplexity(unknown_handling(real_validation_combined), real_model, 1))
print('perplexity of bigram model with real validation file', perplexity(unknown_handling(real_validation_combined), real_model2, 2))
print('perplexity of unigram model with fake validation file', perplexity(unknown_handling(fake_validation_combined), fake_model, 1))
print('perplexity of bigram model with fake validation file', perplexity(unknown_handling(fake_validation_combined), fake_model2, 2))

real_pp = []
fake_pp = []
real_pp2 = []
fake_pp2 = []
for news in real_validation_sep:
  real_pp.append(perplexity(news, real_model, 1))
  real_pp2.append(perplexity(news, real_model2, 2))
for news in fake_validation_sep:
  fake_pp.append(perplexity(news, fake_model, 1))
  fake_pp2.append(perplexity(news, fake_model2, 2))
print('first ten outputs of unigram model with real news validation file', real_pp[:10])
print('first ten outputs of bigram model with real news validation file', real_pp2[:10])
print('first ten outputs of unigram model with fake news validation file', fake_pp[:10])
print('first ten outputs of bigram model with fake news validation file', fake_pp2[:10])

def predict(news_lst, real_model, fake_model, mode):
  prediction = []
  if mode == 1:
    for news in news_lst:
      pp_real = perplexity(news, real_model, mode)
      pp_fake = perplexity(news, fake_model, mode)
      if pp_real < pp_fake:
        prediction.append(1)
      else:
        prediction.append(0)
  if mode == 2:
    for news in news_lst:
      pp_real = perplexity(news, real_model, mode)
      pp_fake = perplexity(news, fake_model, mode)
      if pp_real < pp_fake:
        prediction.append(1)
      else:
        prediction.append(0) 
  return prediction

print('first ten outputs of unigram model with real news validation file', predict(real_validation_sep, real_model, fake_model, 1)[:10])
print('first ten outputs of unigram model with fake news validation file', predict(fake_validation_sep, real_model, fake_model, 1)[:10])
print('first ten outputs of bigram model with real news validation file', predict(real_validation_sep, real_model2, fake_model2, 2)[:10])
print('first ten outputs of bigram model with fake news validation file', predict(fake_validation_sep, real_model2, fake_model2, 2)[:10])

perplexity of unigram model with real validation file 1006.022759643955
perplexity of bigram model with real validation file 633.1890539844053
perplexity of unigram model with fake validation file 1122.1353106785102
perplexity of bigram model with fake validation file 1308.6974751714697
first ten outputs of unigram model with real news validation file [805.6725366779382, 777.6246449382455, 839.4794081675699, 745.5962992054159, 982.085648809443, 1050.5249973395162, 846.3485418581442, 924.661604301053, 1059.4787454062962, 838.3922223265982]
first ten outputs of bigram model with real news validation file [553.0712316752283, 487.6738060057743, 423.70491235988516, 454.445281911637, 719.9978024209249, 576.2010408642599, 439.73774310217397, 617.7535581850664, 584.1472230908762, 514.7458515030386]
first ten outputs of unigram model with fake news validation file [1159.1511184423796, 1150.7474396103207, 789.308207555914, 1079.8205865935088, 866.2658445745495, 1657.4858015158925, 800.6046204964

**Q4.1: Why do we need to compute perplexity after smoothing?**

Your answer: Because there are cases that probabilities of some tokens are 0, which will lead to calculation error. We cannot divide 0.

**Q4.2: Did you choose any values for parameters?**

Your answer: We choose parameter k as 1 for add-k smooth.

## Task 2: Compute perplexity for other smoothing methods (optional). 
*Note: If you only pick one smoothing method, you can omit this task. If you need to try different values of parameters, you can try them out here.*

In [ ]:
# TODO: compute perplexity for your rest of smoothing method.

**Q4.3: If your smoothing method needs to pick a parameter, what is the value of your parameter?**

Your answer: We choose parameter k as 1 for add-k smooth.

**Q4.4: Which smoothing method is the best among your choices?**

Your answer: smoothed bigram model is best since it has lowest perplexity.

# Part 5: Putting Everything Together and Submitting to Kaggle
Combining all the previous parts together, we have developed a bunch of language models. Before we proceed to the next step, let's check a few things (no need to answer):
* Did you train your model only on training set?
* Did you validate your model only on validation/development set?
* Did you determine all your parameters?

Finally, please answer:

**Q5: What is your choice of language model, and why?**

Your answer: We choose bigram smoothed model becasue bigram model has lower perplexity than unigram model, which means a better model.



In [ ]:
uni_real = perplexity(real_validation_sep[0], real_model, 1)
bi_real = perplexity(real_validation_sep[0], real_model2, 2)
print(uni_real > bi_real)

True


## Part 5.1: First Model Submission to Kaggle

Now we need to apply our model to testing data. What you need to do:
* Takes the test data as input, and generates an output of your prediction based on your chosen language model
* Your output file should be ONLY your predictions
* Submit to Kaggle

You should use your trained model to predict labels for all the news in `TestData.txt`. Output your predictions to a **csv** file and submit it to kaggle. Each line should contain the id of the test news and its corresponding prediction (in total 4489 lines). In other words, your output should look like (**including the header**):
```
Id,Prediction
0,0
1,0
2,1
3,0
...
4488,1
```
Note that you should add the header `Id,Prediction` and there is no space in the output. The Id starts from 0 (not 1).

Use this kaggle [link](https://www.kaggle.com/t/a032d54898b24b4bb2f29e7165ac5fdf) to submit your output. Your team name should be the concatenation of your netids, **exactly in the same order as this notebook is named**. For example, if notebook is 4740_FA20_p1_jb123_cj456, then Kaggle group should be jb123_cj456.


In [ ]:
import csv
from google.colab import files

with io.open(os.path.join(dataset_path, "TestData.txt"), encoding='utf8') as test_file:
  test_news = test_file.read()

test_para_list = test_news.splitlines()
for para in range(len(test_para_list)):
  test_para_list[para] = ['startsymbol ' + sub for sub in sent_tokenize(test_para_list[para])] 
  test_para_list[para] = ' '.join(test_para_list[para])
test_sep = [list(map(str.lower, word_tokenize(sent))) for sent in test_para_list]
for para in range(len(test_sep)):
  for w in range(len(test_sep[para])):
    if test_sep[para][w] == 'startsymbol':
      test_sep[para][w] = '<s>'

fields = 'Id,Prediction\n'
filename = "yc2668_zc436.csv"
    
with open(filename, 'w') as csvfile:   
  csvfile.write(fields)
  idx = 0
  for sent in test_sep:
    pp_real = perplexity(sent, real_model, 1)
    pp_fake = perplexity(sent, fake_model, 1)
    if pp_real < pp_fake:
      csvfile.write(str(idx) + ',1\n')
    else:
      csvfile.write(str(idx) + ',0\n')
    idx += 1

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Part 6: Naive Bayes

The Naive Bayes classification method is based on Bayes Rule. Suppose we have a news article *d* and its label *c* (either 0 or 1).
\begin{align*}
P(c|d)=\frac{P(d|c)P(c)}{P(d)}
\end{align*}
Likelihood: $P(d|c)$. In real/fake corpus, how likely *d* would appear.

Prior: $P(c)$. The probability of real/fake news in general.

Posterior: $P(c|d)$. Given *d*, how likely is it that it is real/fake.

Goal: $\underset{c\in \{0,1\}}{\operatorname{argmax}} P(c|d)$, which is equivalent to $\underset{c\in \{0,1\}}{\operatorname{argmax}} P(d|c)P(c)$.

The equivalence holds because $P(d)$ is the same for any $c$. Thus the denominator can be dropped.

Denote $d=\{x_1, x_2, ..., x_n\}$ where $x_i$'s are words in the news *d* (sometimes called features). Unlike n-gram language modelling, we make the multinomial Naive Bayes independence assumption here, where we assume positions of words do not matter. Formally, 
\begin{align*}
&\underset{c\in \{0,1\}}{\operatorname{argmax}} P(d|c)P(c)\\
=&\underset{c\in \{0,1\}}{\operatorname{argmax}} P(x_1, ..., x_n|c)P(c)\\
=&\underset{c\in \{0,1\}}{\operatorname{argmax}} P(x_1|c)P(x_2|c)...P(x_n|c)
\end{align*}

Now we only need to collect the occurences of each word for the classification. This is often called a **bag of words** feature. 

For instance, in the sentence `All for one and one for all .`, the bag of words feature would be `{"all": 2, "for": 2, "one": 2, "and": 1, ".": 1}`. Essentially, the bag of words feature is a dictionary which maps the word to its occurences. We can see that the order is not considered here.

Now, your goal is to implement the Multinomial Naive Bayes. You can use existing codes or Python packages, and adapt them to our news classification task.

You might find the following packages/functions useful:

* nltk.word_tokenize(), nltk.word_tokenize()
* nltk.classify.naivebayes()
* sklearn.feature_extraction.text
* sklearn.naive_bayes.MultinomialNB()

**Please answer the following question(s).**

**Q6: Comparing Multinomial Naive Bayes with the unigram language model, which one do you expect to perform better? Why?**

Your answer: We expect Multinomial Naive Bayes because it takes classes of words into consideration before calculating the probability, which is similar with bigram model in which preceeding word is considered.


## 6.1 Implementation

In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
tfidf_transformer = TfidfTransformer()
count_vect = CountVectorizer()
reals = real_news.splitlines()
fakes = fake_news.splitlines()
total_data = reals + fakes
real_validation = real_news_validation.splitlines()
fake_validation = fake_news_validation.splitlines()
feature = ['Real'] * len(reals) + ['Fake'] * len(fakes)
X_train_counts = count_vect.fit_transform(total_data)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, feature)

## 6.2 Putting Everything Together and Submitting to Kaggle

You should use your trained model to predict labels for all the news in `TestData.txt`. Output your predictions to a **csv** file and submit it to kaggle. The format should follow Part 6 as well.

Use this Kaggle [link](https://www.kaggle.com/t/e8e4f7ee507843d1902c168529d705c8) to submit your output.

In [ ]:
import csv
from google.colab import files

test_para_list = test_news.splitlines()
test_counts = count_vect.transform(test_para_list)
test_tfidf = tfidf_transformer.transform(test_counts)
predicted = clf.predict(test_tfidf)

fields = 'Id,Prediction\n'
filename = "yc2668_zc436.csv"

with open(filename, 'w') as csvfile:   
 csvfile.write(fields)
 idx = 0
 for result in predicted:
   if result == 'Real':
     csvfile.write(str(idx) + ',1\n')
   else:
     csvfile.write(str(idx) + ',0\n')
   idx += 1
     
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Work Distribution

**Please briefly describe how you divided the work.**

Your answer: We have a great cooperation here. Zeyu finished data preprocessing, unigran and bigram model, unknown words and perplexity. Yuanhong helped with perplexity, and finished naive bayes classification. We both finished following questions. 


# Project Feedback [1 point]
It goes without saying that this is an unprecedented time. We on the course staff are trying our best to adapt our teaching, projects and everything else in the class to this new modality. We would immenselly appreciate it if you could provide us feedback (it's a super short form!!) on this project and **it's worth 1 point of your project grade**

Link to the feedback form: https://forms.gle/xfonP9rJk9Uk7zaSA 

We will use this feedback to improve both **upcoming projects** and projects for next year. 

Thank you so much!

# Submitting the Notebook

1. Go to File (upper left corner) -> Download .ipynb -> submit this downloaded file to cms
2. Run the first code block
3. Replace our placeholder for your correct Google Drive directory structure in the 2nd code block below. Run the code block
4. Put the name of this notebook into our placeholder in the 3rd code block. Run the code block
5. Then go to the folder icon on the very left panel, under the orange CO logo. Click on the folder and wait for a PDF version of your notebook to appear. Might take a few minutes.
6. Download the pdf version and submit to Gradescope

In [ ]:
%%capture
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [ ]:
%%capture
# the red text is a placeholder! Change it to your directory structure!
!cp 'drive/My Drive/CS 4740/Project 1/4740_FA20_p1_yc2668_zc436.ipynb' ./ 

In [ ]:
# the red text is a placeholder! Change it to the name of this notebook!
!jupyter nbconvert --to PDF "4740_FA20_p1_yc2668_zc436.ipynb"

[NbConvertApp] Converting notebook 4740_FA20_p1_yc2668_zc436.ipynb to PDF
[NbConvertApp] Writing 97534 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 105256 bytes to 4740_FA20_p1_yc2668_zc436.pdf


# You are done! ✅